In [ ]:
import json
from copy import deepcopy

from hscap.constants import *
from hscap.data import read_hcris_gj

# Generate CCM facility data

The CovidCareMap (CCM) facility data describes a facility's capacity in 3 different scenarios: Conventional, Contingency and Crisis.

Methods taken from [1].

[1] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4636910/#R20

In [ ]:
with open('../data/dh_hcris_merged_facility_data.geojson') as f:
    geojson = json.loads(f.read())

In [ ]:
geojson['features'][1]

## Methods 

We use the methods in [1], but instead of starting with staffing numbers and working towards calculating "# of patients treatable", we work backwards and determine staffing requirements based on the # of patients being treated by a hospital that is saturated based on the ICU bed available determined by the Level of Care scenario.

![Table 1 from (1)](images/table-1-capacity-estimation.png)
*Figure from [1]*

### Scenario ICU Beds Available

For each scenario, we estimate the total number of beds at the facility that will be used as an ICU bed (whether it is a true ICU bed or not), and use an occupancy rate to determine a surge percentage. This differs from the methodology in [1] as they use a surge percentage of 10% and 20% in their estimates. Here we use the equation `100% - {Occupancy Rate %}` to estimate the number of beds available for use in a PHE. Similar to [1], we tansfer this estimate to staffing needs. If no occupancy data is available for the facility, the 10% figure from [1] is used.

The total ICU beds considered vary by the level of care scenario (see below).

### Ventilators Needed

We assume each licensed ICU bed has a ventilator. We therefore calcululate the number of ventilators needed by the total ICU beds considered for the scenario minus the total licensed ICU beds.

We estimate the total licensed ICU beds based on the ratio of total licensed beds to total staffed beds applied to the known staffed ICU beds. **Note:** This methodology should be improved. One idea is to collect as much known licensed ICU bed counts from state data sources as possible and create estimates there from the known staffed ICU bed counts.

### Staff ratios

The patient-per-staff ratio and shifts-per-day are directly taken from [1].

### Parameters by Scenario

#### Conventional

Total IU beds are determined using the ICU Staffed beds counts in the data.
Available percentage is calculated using the ICU Occupance Rate information in the data.

#### Contingency

Total ICU beds are estimated by using the ratio of the US-wide counts in [1]. Availability is calculated using the same rates as the Conventional scenario.

#### Crisis

Total ICU beds are determined to be total licensed beds - every bed in the hospital is converted to an ICU bed. This would require more mechanical ventilators than the facility
is assumed to have, which will be reflected in the estimated ventilators required for
this scenario. The availability rate is calculated as the weighted average of the
ICU occupancy rate and total occupancy rate duing Business as Usual (BAU), with the weights
being based on the ratio of staffed ICU beds to total staffed beds and staffed non-ICU beds
to total staffed beds.

### Data Assumptions

We use data from [Definitive Health](https://coronavirus-resources.esri.com/datasets/definitivehc::definitive-healthcare-usa-hospital-beds?geometry=125.859%2C-16.820%2C-150.821%2C72.123) and [HCRIS]( https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/Cost-Reports/Hospital-2010-form). Below is a list of assumptions used about the data.

- If there is no staffed ICU bed data available, it is assumed the facility has 0 ICU beds.
- If there is no total staffed bed data available, but there are licensed beds, we assume that all beds are staffed beds.


In [ ]:
# The number of shifts each staff type works per day. Taken from [1]
SHIFTS_PER_DAY = 2

# From national counts in [1]
CONTINGENCY_BED_RATIO = 261790 / 81790.0

def printprops(props):
    print(json.dumps(props, indent=4))

def get_staffed_icu_beds(props):
    staffed_icu_beds = props['ICU Total Staffed Beds']
    if staffed_icu_beds is None:
        staffed_icu_beds = props['NUM_ICU_BE']
        
        if staffed_icu_beds is None:
            # Assume this means there are no ICU beds at this facility
            staffed_icu_beds = 0
    return staffed_icu_beds

def get_total_staffed_beds(props):
    total_staffed_beds = props['Total Staffed Beds']
    if total_staffed_beds is None:
        total_staffed_beds = props['NUM_STAFFE']
        
        if total_staffed_beds is None:
            # Assume that total staffed = total licensed
            total_staffed_beds = get_licensed_beds(props)
        
    return total_staffed_beds

def get_licensed_beds(props):
    # Number of total licensed beds
    licensed_beds = props['NUM_LICENS']
    if licensed_beds is None:
        print(json.dumps(props, indent=4))
        raise Exception('No licensed bed data found for the printed facility')
    return licensed_beds

def construct_scenario_data(props):
        
    staffed_icu_beds = get_staffed_icu_beds(props)

    total_staffed_beds = get_total_staffed_beds(props)
        
    if not 'ICU Occupancy Rate' in props:
        printprops(props)
    
    icu_occupancy_rate = props['ICU Occupancy Rate']
    if icu_occupancy_rate is None:
        icu_occupancy_rate = props['BED_UTILIZ']
        
        if icu_occupancy_rate is None:
            # No occupancy data; assume 10%
            icu_occupancy_rate = 0.1
        
    total_occupancy_rate = props['Total Bed Occupancy Rate']
    if total_occupancy_rate is None:
        total_occupancy_rate = props['BED_UTILIZ']
        
        if total_occupancy_rate is None:
            # No occupancy data; assume 10%
            total_occupancy_rate = 0.1
        
    contingency_icu_beds = staffed_icu_beds * CONTINGENCY_BED_RATIO
        
    # Weighted average based on staffed bed counts and occupancy rates
    crisis_occupancy_rate = \
        ((staffed_icu_beds / total_staffed_beds) * icu_occupancy_rate) + \
            ((1 - (staffed_icu_beds / total_staffed_beds)) * total_occupancy_rate)   
    
    return {
        CONVENTIONAL: {
            BEDS: {
                'available percentage': (1 - icu_occupancy_rate),
                'total': staffed_icu_beds,
            },
            PHYSICIANS: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [10, 15],
            },
            RESP_THERP: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [4, 6],
            },
            NURSE: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [1, 1],
            }
        },
        CONTINGENCY: {
            BEDS: {
                'available percentage': (1 - icu_occupancy_rate),
                'total': contingency_icu_beds,
            },
            PHYSICIANS: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [10, 15],
            },
            RESP_THERP: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [7, 9],
            },
            NURSE: {
                'available percentage': (1 - icu_occupancy_rate),
                'patients per': [2, 2],
            }
        },
        CRISIS: {
            BEDS: {
                'available percentage': (1 - crisis_occupancy_rate),
                'total': staffed_icu_beds,
            },
            PHYSICIANS: {
                'available percentage': (1 - crisis_occupancy_rate),
                'patients per': [24, 24],
            },
            RESP_THERP: {
                'available percentage': (1 - crisis_occupancy_rate),
                'patients per': [4, 6],
            },
            NURSE: {
                'available percentage': (1 - crisis_occupancy_rate),
                'patients per': [1, 1],
            }
        }
    }


In [ ]:
# Properties to map over, by new_property_name -> source_property_name
properties_to_directly_map = {
    'Name': 'HOSPITAL_N',
    'Hospital Type': 'HOSPITAL_T',
    'Address': 'HQ_ADDRESS',
    'Address_2': 'HQ_ADDRE_1',
    'City': 'HQ_CITY',
    'State': 'HQ_STATE',
    'Zipcode': 'HQ_ZIP_COD',
    'County': 'COUNTY_NAM',
    'DH_ID': 'OBJECTID',
    'HCRIS_ID': 'Provider Number'
}

In [ ]:
new_gj = deepcopy(geojson)
for feature in new_gj['features']:
    props = feature['properties']
    new_props = {}
    
    scenario_data = construct_scenario_data(props)
    
    # Number of staffed ICU beds durig 'Business As Usual' (BAU)
    staffed_icu_beds_BAU = get_staffed_icu_beds(props)
        
    licensed_beds = get_licensed_beds(props)

    # Total number of staffed beds during BAU
    staffed_beds_BAU = get_total_staffed_beds(props)
    
    # Validation: Highlight if the HCRIS data disagrees with the DH data, if available
    if props['Total Staffed Beds'] is not None:
        hcris_staffed_beds_BAU = props['Total Staffed Beds']
        if abs(hcris_staffed_beds_BAU - staffed_beds_BAU) > 10:
            print('Facility {} ({})has a disagreement about staffed bed numbers!'.format(
                props['HOSPITAL_N', 'OBJECTID']))

    # Estimate the total licensed ICU beds based on total/staffed ratio
    licensed_to_staffed_ratio = (licensed_beds / staffed_beds_BAU) * staffed_beds_BAU
    
    if licensed_to_staffed_ratio is None:
        print('whu')
        print(json.dumps(props, indent=4))
        
    ## TODO: Replace this with something better (as per note in Methods)
    estimated_licensed_icu_beds = licensed_to_staffed_ratio * staffed_icu_beds_BAU
    
        
    for scenario_name, scenario in scenario_data.items():
        icu_beds_total = scenario[BEDS]['total']
        icu_beds = icu_beds_total * scenario[BEDS]['available percentage']       
        
        new_props[construct_beds_field_name(scenario_name)] = round(icu_beds)
        
        # Ventilator need
        new_props[construct_vents_field_name(scenario_name)] = \
            round(max(0, icu_beds - estimated_licensed_icu_beds))
        
        for staff_need in STAFF:
            patients_per = scenario[staff_need]['patients per']
            available_percentage = scenario[staff_need]['available percentage']
            
            if icu_beds <= 0:
                min_need, max_need = 0, 0
            else:
                min_need, max_need = None, None
            
                for patients in patients_per:
                    need_count = (icu_beds * available_percentage) / patients
                    need_count /= SHIFTS_PER_DAY
                    
                    if min_need is None or need_count < min_need:
                        min_need = need_count
                    if max_need is None or max_need < need_count:
                        max_need = need_count
                        
            new_props[construct_staff_field_name(
                scenario_name, staff_need, BOUND_LOWER)] = round(max(1, min_need))
            new_props[construct_staff_field_name(
                scenario_name, staff_need, BOUND_UPPER)] = round(max(1, max_need))
            
    for new_prop, old_prop in properties_to_directly_map.items():
        new_props[new_prop] = props[old_prop]
    
    feature['properties'] = new_props
    
print(json.dumps(new_gj['features'][0], indent=4))

In [ ]:
open('../data/ccm_data_by_facility.geojson', 'w').write(json.dumps(new_gj, indent=2))